In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import sys
import xarray as xr
import matplotlib.patches as mpatches
import os
from lib import conversion

# plot des zones d'intérêts pour l'agrégation 

In [6]:
dir_fig = '../../../figures/bilan_agregation/'
dir_csv = '../zonageWME/'

# date = '2020012600'
date = '2020030600'
echeance = 30 
dep_id = '41'
index_zone = 5

if date == '2020012600':
    echeance_dict = {
        '38':[3,3],
        '29':[13,13],  
        '34':[6,30,1,4,4], 
        '41':[30,5]
    }
    index_zone_dict ={
        '41':[2,0],
        '34':[0,0,0,0,1],
        '38':[0,1],
        '29':[0,2]
    }

if date == '2020030600':
    echeance_dict = {
        '38':[3,29],
        '29':[5],  
#         '34':[], 
#         '41':[]
    }
    index_zone_dict ={
#         '41':[],
#         '34':[],
        '38':[0,1],
        '29':[0]
    }


In [7]:
''' lecture arome '''
for dep_id in echeance_dict.keys():
    echeance_list = echeance_dict[dep_id]
    fname = "../WWMF/" + date+'0000__PG0PAROME__'+'WWMF'+'__EURW1S100______GRILLE____0_48_1__SOL____GRIB2.nc'
    print(fname)

    ds = xr.open_dataset(fname,chunks={"step":1}).isel(step = echeance_list)
    ds['latitude']  = ds['latitude'].round(5)
    ds['longitude'] = ds['longitude'].round(5)
    print(echeance_list)
    '''lecture masque'''
    fname_out = '../GeoData/zones_sympo_multiples/'+dep_id+'_mask_zones_sympos.nc'
    if os.path.exists(fname_out): 
        ds_mask = xr.open_dataset(fname_out,chunks={"id":1})
        ds_mask["latitude"]  = ds_mask["latitude"].round(5)
        ds_mask["longitude"] = ds_mask["longitude"].round(5)

    for icheance,echeance in enumerate(echeance_dict[dep_id]):
        index_zone = index_zone_dict[dep_id][icheance]
        ''' lecture fichier resultat '''
        fname_csv_ini = dir_csv+ 'v2_'+dep_id+'_'+date+'_'+str(echeance)+'.csv'
        df = pd.read_csv(fname_csv_ini)
        id_zone = df.iloc[index_zone]['zone']

        ds_dep_tot = (ds*ds_mask.mask.sel(id=id_zone).drop("id"))
        if date == '2020030600':
            ds_dep_tot = ds_dep_tot.rename({'paramId_0':'unknown'})
        # conversion
        for name in ['compas','agat']:
            ds_dep_tot = conversion (ds_dep_tot,name)
            
        # label pour le plot
        label = 'compas:{}, compas asym: {}, agat:{}, agat asym: {}'.format(df.iloc[index_zone]['compas'],
                                df.iloc[index_zone]['compas_asym'],df.iloc[index_zone]['agat'],
                                df.iloc[index_zone]['agat_asym'])

        fig,axes = plt.subplots(nrows = 1, ncols=3,figsize  = (15,5))
        ax       = axes.flat
        var2plot_lst = ['unknown','wme_arr','w1_arr']
        title_lst    = ['WWMF','WME','W1']
        varmin_lst   = [0,1,0]
        varmax_lst   = [99,19,30]
        for iplot in range(3):
            var2plot = ds_dep_tot[var2plot_lst[iplot]].isel(step = 0).where(ds_mask.mask.sel(id=id_zone)==1,drop=True)
            if iplot == 0 : 
                cmap  = matplotlib.cm.jet
            else: 
                cmap = matplotlib.cm.tab20b

            varmin   = varmin_lst[iplot]
            varmax   = varmax_lst[iplot] + 1        
            clevs    = np.arange(varmin,varmax+1,1)
            cs       = var2plot.plot.imshow(ax = ax[iplot],cmap=cmap,levels=clevs,cbar_kwargs={'label':title_lst[iplot]}) #cbar_kwargs 
            ax[iplot].set_title('')
        fig.suptitle(label,y=1.05,fontsize = 18)
        fig.tight_layout()
        fig.savefig(dir_fig+dep_id+'_'+date+'_'+str(echeance)+'_'+str(index_zone)+'.png',bbox_inches='tight',dpi=400)
        plt.clf()
        plt.close('all')

../WWMF/20200306000000__PG0PAROME__WWMF__EURW1S100______GRILLE____0_48_1__SOL____GRIB2.nc
[3, 29]
../WWMF/20200306000000__PG0PAROME__WWMF__EURW1S100______GRILLE____0_48_1__SOL____GRIB2.nc
[5]


In [4]:

# label

In [5]:
sys.exit()

SystemExit: 

C:\Users\mary\anaconda3\envs\preproc\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
file_CodesWWMF = '../utils/CodesWWMF.csv'
df             = pd.read_csv(file_CodesWWMF,usecols = (0,1,2,3,6,7),sep=',')
Code_WME       = df['Code WME'].to_numpy()
Code_WWMF      = df['Code WWMF'].to_numpy()
Code_W1        = df['Code W1'].to_numpy()
legende_WWMF   = df['Legende WWMF'].to_numpy()
legende_WME    = df['Legende WME'].to_numpy()
legende_W1     = df['Legende W1'].to_numpy()

Code_W1.max()

In [ ]:
code_cible = 3

legende_WME_cible = np.unique(legende_WME[Code_WME==code_cible])
legende_W1_cible  = np.unique(legende_W1[Code_WME==code_cible])

print(legende_WME_cible)
print(legende_W1_cible)